In [1]:
from googleapiclient.discovery import build
import pandas as pd

In [2]:
api_key = 'xxx'
channel_id = 'xxx'

youtube = build('youtube','v3',developerKey=api_key)

## Function to get channel statistics

In [3]:
def get_channel_stats(youtube,channel_id):
    
    request = youtube.channels().list(
                                     part='snippet,contentDetails,statistics',
                                     id=channel_id)
    response = request.execute()
    
    data = dict(Channel_name = response['items'][0]['snippet']['title'],
               Subscribers = response['items'][0]['statistics']['subscriberCount'],
               Views = response['items'][0]['statistics']['viewCount'],
               Total_videos = response['items'][0]['statistics']['videoCount'],
               playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads'])
    
    return data

In [4]:
get_channel_stats(youtube,channel_id)

{'Channel_name': 'S Sport',
 'Subscribers': '1130000',
 'Views': '227818009',
 'Total_videos': '9702',
 'playlist_id': 'UUpdSUUHlxMjO0c5824FGcsA'}

In [5]:
channel_statistics = get_channel_stats(youtube,channel_id)

In [6]:
channel_data = pd.DataFrame(channel_statistics,index=[0])

In [7]:
channel_data

,Channel_name,Subscribers,Views,Total_videos,playlist_id
0,S Sport,1130000,227818009,9702,UUpdSUUHlxMjO0c5824FGcsA


In [8]:
channel_data.dtypes

Channel_name    object
Subscribers     object
Views           object
Total_videos    object
playlist_id     object
dtype: object

In [9]:
channel_data['Subscribers'] = pd.to_numeric(channel_data['Subscribers'])
channel_data['Views'] = pd.to_numeric(channel_data['Views'])
channel_data['Total_videos'] = pd.to_numeric(channel_data['Total_videos'])

In [10]:
channel_data.dtypes

Channel_name    object
Subscribers      int64
Views            int64
Total_videos     int64
playlist_id     object
dtype: object

In [25]:
channel_data.to_csv('csv/channel_data_04-11-22.csv',index=False)

In [14]:
channel_statistics['playlist_id']

'UUpdSUUHlxMjO0c5824FGcsA'

In [15]:
playlist_id = channel_statistics['playlist_id']

## Function to get video ids

In [16]:
def get_video_ids(youtube,playlist_id):
    
    request = youtube.playlistItems().list(
                                          part = 'contentDetails',
                                          playlistId = playlist_id,
                                          maxResults = 50)
    response = request.execute()
    
    video_ids = []
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:         
            request = youtube.playlistItems().list(
                                          part = 'contentDetails',
                                          playlistId = playlist_id,
                                          maxResults = 50,
                                          pageToken = next_page_token)
            response = request.execute()
            
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
            next_page_token = response.get('nextPageToken')
    
    return video_ids

In [17]:
video_ids = get_video_ids(youtube,playlist_id)

In [18]:
len(video_ids)

9698

## Function to get video details

In [19]:
def get_video_details(youtube,video_ids):
    
    all_video_stats = []
    
    for i in range(0,len(video_ids),50):
        request = youtube.videos().list(
                                        part = 'snippet,statistics,contentDetails,id',
                                        id = ','.join(video_ids[i:i+50])
                                       )
        response = request.execute()
        try:
            for video in response['items']:
                video_stats = dict(Id = video['id'],
                              Title = video['snippet']['title'],
                              Published_date = video['snippet']['publishedAt'],
                              Views = video['statistics']['viewCount'],
                              Likes = video['statistics']['likeCount'],
                              Comments = video['statistics']['commentCount'],
                              Duration = video['contentDetails']['duration']
                              )
                all_video_stats.append(video_stats)
        except:
            pass
    
    
    return all_video_stats

In [20]:
video_details = get_video_details(youtube,video_ids)

In [21]:
video_data = pd.DataFrame(video_details)

In [22]:
video_data

,Id,Title,Published_date,Views,Likes,Comments,Duration
0,V0jlhEAScls,NBA I En İyi Oyunlar,2022-11-04T07:00:11Z,341,14,0,PT1M18S
1,EE9j8Vjslcs,Denver Nuggets - Oklahoma City Thunder Maç Öze...,2022-11-04T06:38:33Z,124,4,0,PT1M36S
2,2Mo1uJpiBl4,Golden State Warriors - Orlando Magic Maç Özet...,2022-11-04T06:38:30Z,393,7,2,PT2M3S
3,vN2q_eycKWw,Olimpia Milano - Real Madrid (77-83) - Maç Öze...,2022-11-03T22:12:26Z,4713,70,8,PT9M9S
4,y2mZrITfk7c,Bayern Münih - Anadolu Efes (81-78) - Maç Özet...,2022-11-03T21:48:40Z,22085,299,99,PT8M19S
...,...,...,...,...,...,...,...
9405,IvcdLnQOaJo,Yoel Romero'nun Luke Rockhold'u Nakavt Ettiği ...,2018-03-09T12:17:56Z,1837,21,5,PT34S
9406,_Q_uV0PuPB0,Jessica Andrade - Tecia Torres Dövüşünde Gecen...,2018-03-09T12:17:55Z,1147,12,1,PT25S
9407,btVGCtXxKzg,Marc Marquez'in Şampiyonluk Heyecanı! (MotoGP ...,2018-03-09T12:17:55Z,353,4,4,PT2M7S
9408,xQgGHq76X7Q,Marquez Hata Yaptı Ama... (MotoGP 2017 - Valen...,2018-03-09T12:17:55Z,1105,13,3,PT34S


In [27]:
video_data.to_csv('csv/video_data_04-11-22.csv',index=False)

In [28]:
pd.read_csv('csv/video_data_04-11-22.csv')

,Id,Title,Published_date,Views,Likes,Comments,Duration
0,V0jlhEAScls,NBA I En İyi Oyunlar,2022-11-04T07:00:11Z,341,14,0,PT1M18S
1,EE9j8Vjslcs,Denver Nuggets - Oklahoma City Thunder Maç Öze...,2022-11-04T06:38:33Z,124,4,0,PT1M36S
2,2Mo1uJpiBl4,Golden State Warriors - Orlando Magic Maç Özet...,2022-11-04T06:38:30Z,393,7,2,PT2M3S
3,vN2q_eycKWw,Olimpia Milano - Real Madrid (77-83) - Maç Öze...,2022-11-03T22:12:26Z,4713,70,8,PT9M9S
4,y2mZrITfk7c,Bayern Münih - Anadolu Efes (81-78) - Maç Özet...,2022-11-03T21:48:40Z,22085,299,99,PT8M19S
...,...,...,...,...,...,...,...
9405,IvcdLnQOaJo,Yoel Romero'nun Luke Rockhold'u Nakavt Ettiği ...,2018-03-09T12:17:56Z,1837,21,5,PT34S
9406,_Q_uV0PuPB0,Jessica Andrade - Tecia Torres Dövüşünde Gecen...,2018-03-09T12:17:55Z,1147,12,1,PT25S
9407,btVGCtXxKzg,Marc Marquez'in Şampiyonluk Heyecanı! (MotoGP ...,2018-03-09T12:17:55Z,353,4,4,PT2M7S
9408,xQgGHq76X7Q,Marquez Hata Yaptı Ama... (MotoGP 2017 - Valen...,2018-03-09T12:17:55Z,1105,13,3,PT34S
